SETUP AND PREPROCESSING- MERGING TWO DATASETS

In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
telemetry_path = "/content/drive/MyDrive/ds677/final_project/dataset/chiller6_june2020_sensordata_couchdb.csv"
anomaly_path = "/content/drive/MyDrive/ds677/final_project/dataset/anomaly_events.csv"


In [ ]:
anom = pd.read_csv(anomaly_path)
tele = pd.read_csv(telemetry_path)


In [ ]:
tele.columns


Index(['asset_id', 'timestamp',
       'Chiller 6 Condenser Water Return To Tower Temperature',
       'Chiller 6 Chiller Efficiency', 'Chiller 6 Tonnage',
       'Chiller 6 Supply Temperature', 'Chiller 6 Return Temperature',
       'Chiller 6 Condenser Water Flow', 'Chiller 6 Schedule',
       'Chiller 6 Power Input', 'Chiller 6 Chiller % Loaded',
       'Chiller 6 Liquid Refrigerant Evaporator Temperature',
       'Chiller 6 Setpoint Temperature', 'Chiller 6 Run Status'],
      dtype='object')

In [ ]:
anom.columns

Index(['timestamp', 'KPI', 'asset_name', 'value', 'upper_bound', 'lower_bound',
       'anomaly_score'],
      dtype='object')

In [ ]:
anom = anom.rename(columns={'asset_name': 'asset_id'})


In [ ]:
tele['timestamp'] = pd.to_datetime(tele['timestamp'])
anom['timestamp'] = pd.to_datetime(anom['timestamp'], errors='coerce')


/tmp/ipython-input-2110644038.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  anom['timestamp'] = pd.to_datetime(anom['timestamp'], errors='coerce')


In [ ]:
anom['asset_id'] = 'chiller 6'


In [ ]:
anom['timestamp'] = anom['timestamp'].dt.floor('15min')


In [ ]:
anom['anomaly_flag'] = 1


In [ ]:
merged = tele.merge(
    anom[['timestamp', 'anomaly_flag']],
    on='timestamp',
    how='left'
)

merged['anomaly_flag'] = merged['anomaly_flag'].fillna(0).astype(int)


In [ ]:
merged['anomaly_flag'].value_counts()


,count
anomaly_flag,
0,2876
1,30


In [ ]:
merged.columns


Index(['asset_id', 'timestamp',
       'Chiller 6 Condenser Water Return To Tower Temperature',
       'Chiller 6 Chiller Efficiency', 'Chiller 6 Tonnage',
       'Chiller 6 Supply Temperature', 'Chiller 6 Return Temperature',
       'Chiller 6 Condenser Water Flow', 'Chiller 6 Schedule',
       'Chiller 6 Power Input', 'Chiller 6 Chiller % Loaded',
       'Chiller 6 Liquid Refrigerant Evaporator Temperature',
       'Chiller 6 Setpoint Temperature', 'Chiller 6 Run Status',
       'anomaly_flag'],
      dtype='object')

In [ ]:
merged.head()


,asset_id,timestamp,Chiller 6 Condenser Water Return To Tower Temperature,Chiller 6 Chiller Efficiency,Chiller 6 Tonnage,Chiller 6 Supply Temperature,Chiller 6 Return Temperature,Chiller 6 Condenser Water Flow,Chiller 6 Schedule,Chiller 6 Power Input,Chiller 6 Chiller % Loaded,Chiller 6 Liquid Refrigerant Evaporator Temperature,Chiller 6 Setpoint Temperature,Chiller 6 Run Status,anomaly_flag
0,Chiller 6,2020-06-01 00:00:00,48.881896,0.289465,451.808092,46.023055,34.639363,6047.885045,1.290502,136.368484,97.295583,13.145167,14.068846,NaN,0
1,Chiller 6,2020-06-01 00:15:00,135.551084,0.304139,937.310395,30.601312,57.643732,8082.333651,NaN,698.487894,109.219081,16.289419,88.996522,NaN,0
2,Chiller 6,2020-06-01 00:30:00,19.053974,0.419444,4.040766,76.403888,105.448163,1864.653705,NaN,496.833518,79.662681,73.134542,50.884664,NaN,0
3,Chiller 6,2020-06-01 00:45:00,57.890020,0.168887,187.825020,43.400981,63.107078,3565.403505,NaN,738.792507,87.382456,51.121515,9.045935,NaN,0
4,Chiller 6,2020-06-01 01:00:00,28.645217,0.144900,1112.909644,77.363087,15.112031,9170.114468,NaN,536.975166,63.676347,30.397869,9.237314,NaN,0


In [ ]:
merged.to_csv("merged_chiller6_with_anomalies.csv", index=False)


In [ ]:
from google.colab import files
files.download("merged_chiller6_with_anomalies.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# copy for ML (drop timestamp)
merged_clean_ml = merged.drop(columns=[
    'asset_id',
    'timestamp',
    'Chiller 6 Schedule',
    'Chiller 6 Run Status'
])

# copy for LLM (keep timestamp)
merged_clean_with_time = merged.drop(columns=[
    'asset_id',
    'Chiller 6 Schedule',
    'Chiller 6 Run Status'
])


In [ ]:
merged_clean_ml.to_csv("merged_clean_ml.csv", index=False)
merged_clean_with_time.to_csv("merged_clean_with_time.csv", index=False)


In [ ]:
from google.colab import files
files.download("merged_clean_ml.csv")
files.download("merged_clean_with_time.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("merged_clean_with_time.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
merged_clean_ml.isna().sum()


,0
Chiller 6 Condenser Water Return To Tower Temperature,20
Chiller 6 Chiller Efficiency,20
Chiller 6 Tonnage,20
Chiller 6 Supply Temperature,21
Chiller 6 Return Temperature,21
Chiller 6 Condenser Water Flow,20
Chiller 6 Power Input,21
Chiller 6 Chiller % Loaded,20
Chiller 6 Liquid Refrigerant Evaporator Temperature,20
Chiller 6 Setpoint Temperature,20


In [ ]:
merged_clean_ml = merged_clean_ml.fillna(merged_clean_ml.median())


In [ ]:
merged_clean_ml.isna().sum()


,0
Chiller 6 Condenser Water Return To Tower Temperature,0
Chiller 6 Chiller Efficiency,0
Chiller 6 Tonnage,0
Chiller 6 Supply Temperature,0
Chiller 6 Return Temperature,0
Chiller 6 Condenser Water Flow,0
Chiller 6 Power Input,0
Chiller 6 Chiller % Loaded,0
Chiller 6 Liquid Refrigerant Evaporator Temperature,0
Chiller 6 Setpoint Temperature,0


In [ ]:
y = merged_clean_ml['anomaly_flag'].values
X = merged_clean_ml.drop(columns=['anomaly_flag']).values


SCALING

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


WINDOWING AND SEQUENCING

In [ ]:
import numpy as np

seq_len = 60
X_seq = []
y_seq = []

for i in range(len(X_scaled) - seq_len):
    X_seq.append(X_scaled[i:i+seq_len])
    y_seq.append(y[i+seq_len])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)


TRAIN -TEST SPLIT AND SAVING THE DATASET

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq
)


In [ ]:
X_train.shape, X_test.shape
y_train.sum(), y_test.sum()


(np.int64(24), np.int64(6))

In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

print("Anomalies in y_train:", y_train.sum())
print("Anomalies in y_test:", y_test.sum())


X_train shape: (2276, 60, 10)
X_test shape: (570, 60, 10)
Anomalies in y_train: 24
Anomalies in y_test: 6


In [ ]:
print(len(merged_clean_ml))
print(len(X_scaled), len(y))
print("Number of sequences:", len(X_seq))


2906
2906 2906
Number of sequences: 2846


In [ ]:
import numpy as np

np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)


In [ ]:
import pickle

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


In [ ]:
from google.colab import files

files.download("X_train.npy")
files.download("X_test.npy")
files.download("y_train.npy")
files.download("y_test.npy")
files.download("scaler.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
np.save("/content/drive/MyDrive/ds677/final_project/X_train.npy", X_train)
np.save("/content/drive/MyDrive/ds677/final_project/X_test.npy", X_test)
np.save("/content/drive/MyDrive/ds677/final_project/y_train.npy", y_train)
np.save("/content/drive/MyDrive/ds677/final_project/y_test.npy", y_test)

with open("/content/drive/MyDrive/ds677/final_project/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

X_train = np.load("/content/drive/MyDrive/ds677/final_project/X_train.npy")
X_test  = np.load("/content/drive/MyDrive/ds677/final_project/X_test.npy")
y_train = np.load("/content/drive/MyDrive/ds677/final_project/y_train.npy")
y_test  = np.load("/content/drive/MyDrive/ds677/final_project/y_test.npy")

print("Shapes:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


Shapes:
X_train: (2276, 60, 10)
X_test: (570, 60, 10)
y_train: (2276,)
y_test: (570,)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = {i : class_weights[i] for i in range(len(class_weights))}
print(class_weights)


{0: np.float64(0.5053285968028419), 1: np.float64(47.416666666666664)}


LSTM MODELS- BASELINE- ENHANCED- LIGHTWEIGHT

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(60, 10)))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 64)         │        19,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,649 (123.63 KB)

 Trainable params: 31,649 (123.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    verbose=1
)


Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8743 - loss: 0.3979 - val_accuracy: 0.8456 - val_loss: 0.3489
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8356 - loss: 0.2171 - val_accuracy: 0.8456 - val_loss: 0.3434
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8762 - loss: 0.1897 - val_accuracy: 0.9439 - val_loss: 0.1574
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9093 - loss: 0.1710 - val_accuracy: 0.8649 - val_loss: 0.3165
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8924 - loss: 0.1843 - val_accuracy: 0.9421 - val_loss: 0.1386
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9163 - loss: 0.1650 - val_accuracy: 0.9386 - val_loss: 0.1660
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9290 - loss: 0.1360 - val_accuracy: 0.9544 - val_loss: 0.1394
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9492 - loss: 0.0921 - val_accuracy: 0.9526 - v

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[[539  25]
 [  2   4]]
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       564
           1       0.14      0.67      0.23         6

    accuracy                           0.95       570
   macro avg       0.57      0.81      0.60       570
weighted avg       0.99      0.95      0.97       570



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np

def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    return alpha * tf.pow((1 - p_t), gamma) * bce

model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(60, 10)))
model.add(Dropout(0.3))
model.add(LSTM(16, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(8))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss=focal_loss,
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

model.summary()

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = {i: class_weights[i] for i in range(len(class_weights))}
print(class_weights)

history = model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 60, 32)         │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 60, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 60, 16)         │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 60, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 8)              │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,449 (36.91 KB)

 Trainable params: 9,449 (36.91 KB)

 Non-trainable params: 0 (0.00 B)

{0: np.float64(0.5053285968028419), 1: np.float64(47.416666666666664)}
Epoch 1/25
72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.7810 - loss: 0.0315 - val_accuracy: 0.9895 - val_loss: 0.0032
Epoch 2/25
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9912 - loss: 0.0110 - val_accuracy: 0.9895 - val_loss: 0.0021
Epoch 3/25
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9844 - loss: 0.0118 - val_accuracy: 0.9895 - val_loss: 0.0017
Epoch 4/25
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9918 - loss: 0.0121 - val_accuracy: 0.9895 - val_loss: 0.0011
Epoch 5/25
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9863 - loss: 0.0162 - val_accuracy: 0.9895 - val_loss: 9.3554e-04
Epoch 6/25
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9902 - loss: 0.0101 - val_accuracy: 0.9895 - val_loss: 7.7381e-04
Epoch 7/25
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9846 - loss: 0.0197 - val_accuracy: 0.9895 - val_loss: 8.6196e-04
Epoch 8/25
72/72 ━━━━━━━━━━━

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_prob = model.predict(X_test)

# Lower threshold usually helps recall for anomaly class
threshold = 0.30
y_pred = (y_prob > threshold).astype(int)

print("Threshold:", threshold)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
Threshold: 0.3
[[564   0]
 [  5   1]]
              precision    recall  f1-score   support

           0     0.9912    1.0000    0.9956       564
           1     1.0000    0.1667    0.2857         6

    accuracy                         0.9912       570
   macro avg     0.9956    0.5833    0.6407       570
weighted avg     0.9913    0.9912    0.9881       570



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np

model = Sequential()
model.add(LSTM(16, return_sequences=True, input_shape=(60, 10)))
model.add(Dropout(0.2))
model.add(LSTM(8))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

model.summary()

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_39 (LSTM)                  │ (None, 60, 16)         │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 60, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_40 (LSTM)                  │ (None, 8)              │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,537 (9.91 KB)

 Trainable params: 2,537 (9.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6277 - loss: 0.7154 - val_accuracy: 0.6807 - val_loss: 0.6131
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6595 - loss: 0.6128 - val_accuracy: 0.6544 - val_loss: 0.6182
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6477 - loss: 0.5966 - val_accuracy: 0.6158 - val_loss: 0.5870
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5980 - loss: 0.6387 - val_accuracy: 0.7807 - val_loss: 0.4648
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7894 - loss: 0.4322 - val_accuracy: 0.6930 - val_loss: 0.5745
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7324 - loss: 0.4815 - val_accuracy: 0.6912 - val_loss: 0.5914
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7419 - loss: 0.4552 - val_accuracy: 0.8632 - val_loss: 0.3710
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8624 - loss: 0.3552 - val_accuracy: 0.8632 - v

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_prob = model.predict(X_test)
threshold = 0.30
y_pred = (y_prob > threshold).astype(int)

print("Threshold:", threshold)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Threshold: 0.3
[[357 207]
 [  1   5]]
              precision    recall  f1-score   support

           0     0.9972    0.6330    0.7744       564
           1     0.0236    0.8333    0.0459         6

    accuracy                         0.6351       570
   macro avg     0.5104    0.7332    0.4101       570
weighted avg     0.9870    0.6351    0.7667       570



In [ ]:

inputs = tf.keras.Input(shape=(60,10))
x = LSTM(16, return_sequences=True)(inputs)
x = Dropout(0.2)(x)
x, h_state, c_state = LSTM(8, return_state=True, return_sequences=False)(x)
prob_output = Dense(1, activation='sigmoid')(x)

tiny_lstm_func = tf.keras.Model(inputs, [prob_output, h_state])

# --- Load weights from trained Sequential model ---
tiny_lstm_func.layers[1].set_weights(model.layers[0].get_weights())   # LSTM(16)
tiny_lstm_func.layers[2].set_weights(model.layers[1].get_weights())   # Dropout
tiny_lstm_func.layers[3].set_weights(model.layers[2].get_weights())   # LSTM(8)
tiny_lstm_func.layers[4].set_weights(model.layers[3].get_weights())   # Dense(1)

# --- Predict ---
lstm_prob, lstm_hidden = tiny_lstm_func.predict(X_test)

lstm_prob = lstm_prob.flatten()
lstm_flag = (lstm_prob > 0.30).astype(int)

print(lstm_prob.shape, lstm_hidden.shape)


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
(570,) (570, 8)


In [ ]:
X_test_pred = lstm_ae.predict(X_test)
errors = np.mean((X_test - X_test_pred)**2, axis=(1,2))

threshold = np.percentile(errors, 95)

y_pred = (errors > threshold).astype(int)

from sklearn.metrics import confusion_matrix, classification_report

print("Threshold:", threshold)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step
Threshold: 0.9573617711660325
[[536  28]
 [  5   1]]
              precision    recall  f1-score   support

           0     0.9908    0.9504    0.9701       564
           1     0.0345    0.1667    0.0571         6

    accuracy                         0.9421       570
   macro avg     0.5126    0.5585    0.5136       570
weighted avg     0.9807    0.9421    0.9605       570



TRANSFORMER - BASELINE- LIGTHWEIGHT

In [ ]:
!pip install tensorflow-addons


ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model


In [ ]:
class PositionalEncoding(layers.Layer):
    def __init__(self, seq_len, d_model):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        self.pos_encoding = self.positional_encoding(seq_len, d_model)

    def positional_encoding(self, seq_len, d_model):
        # positions: (seq_len, 1)
        positions = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
        dims = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]

        # angle rates
        angle_rads = positions * (1.0 / tf.pow(10000., (2 * (dims // 2)) / d_model))

        # apply sin to even indices 0,2,4,...
        sines = tf.sin(angle_rads[:, 0::2])

        # apply cos to odd indices 1,3,5,...
        cosines = tf.cos(angle_rads[:, 1::2])

        # interleave sin and cos
        pos_encoding = tf.reshape(
            tf.concat([sines[..., tf.newaxis], cosines[..., tf.newaxis]], axis=-1),
            (seq_len, d_model)
        )

        return pos_encoding[tf.newaxis, ...]

    def call(self, inputs):
        inputs = tf.cast(inputs, tf.float32)
        return inputs + tf.cast(self.pos_encoding, tf.float32)


In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
    # Multi-head self-attention
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed-forward network
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.Dropout(dropout)(ff)
    out = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    return out


In [ ]:
sequence_len = X_train.shape[1]   # 60
features = X_train.shape[2]       # 10

embed_dim = 32
num_heads = 4
ff_dim = 64
dropout_rate = 0.1


In [ ]:
inputs = layers.Input(shape=(sequence_len, features))

# Project features into embedding dimension
x = layers.Dense(embed_dim)(inputs)

# Ensure float32
x = layers.Lambda(lambda t: tf.cast(t, tf.float32))(x)

# Positional encoding (using your class)
x = PositionalEncoding(sequence_len, embed_dim)(x)

# Transformer blocks
x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)

# Pooling + classification head
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)


In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    metrics=["accuracy"]
)


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32,
    class_weight=class_weights,   # imbalance handling
    verbose=1
)


Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 19s 101ms/step - accuracy: 0.7429 - loss: 0.7446 - val_accuracy: 0.6842 - val_loss: 0.6385
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7108 - loss: 0.5579 - val_accuracy: 0.0105 - val_loss: 1.2058
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5072 - loss: 0.6953 - val_accuracy: 0.6719 - val_loss: 0.7062
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6580 - loss: 0.6154 - val_accuracy: 0.6561 - val_loss: 0.7990
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6205 - loss: 0.6502 - val_accuracy: 0.6719 - val_loss: 0.6724
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5870 - loss: 0.6676 - val_accuracy: 0.6737 - val_loss: 0.6231
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6613 - loss: 0.6067 - val_accuracy: 0.7649 - val_loss: 0.4444
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7915 - loss: 0.4995 - val_accuracy: 0.6632 - val_lo

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = (model.predict(X_test) > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))


18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 254ms/step
[[531  33]
 [  3   3]]
              precision    recall  f1-score   support

           0     0.9944    0.9415    0.9672       564
           1     0.0833    0.5000    0.1429         6

    accuracy                         0.9368       570
   macro avg     0.5389    0.7207    0.5550       570
weighted avg     0.9848    0.9368    0.9585       570



In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

thresholds = [0.1, 0.2, 0.3, 0.4, 0.5]

probs = model.predict(X_test)

for t in thresholds:
    preds = (probs > t).astype(int)
    print("\n=== Threshold:", t, "===")
    print(confusion_matrix(y_test, preds))
    print(classification_report(y_test, preds, digits=4))


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

=== Threshold: 0.1 ===
[[502  62]
 [  3   3]]
              precision    recall  f1-score   support

           0     0.9941    0.8901    0.9392       564
           1     0.0462    0.5000    0.0845         6

    accuracy                         0.8860       570
   macro avg     0.5201    0.6950    0.5119       570
weighted avg     0.9841    0.8860    0.9302       570


=== Threshold: 0.2 ===
[[516  48]
 [  3   3]]
              precision    recall  f1-score   support

           0     0.9942    0.9149    0.9529       564
           1     0.0588    0.5000    0.1053         6

    accuracy                         0.9105       570
   macro avg     0.5265    0.7074    0.5291       570
weighted avg     0.9844    0.9105    0.9440       570


=== Threshold: 0.3 ===
[[524  40]
 [  3   3]]
              precision    recall  f1-score   support

           0     0.9943    0.9291    0.9606       564
           1     0.0698    0.5000    0.1224         6

 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

seq_len = X_train.shape[1]   # 60
feature_dim = X_train.shape[2]  # 10

embed_dim = 16
num_heads = 2
ff_dim = 32

inputs = layers.Input(shape=(seq_len, feature_dim))

x = layers.Dense(embed_dim)(inputs)

positions = tf.range(start=0, limit=seq_len, delta=1)
pos_embedding = layers.Embedding(input_dim=seq_len, output_dim=embed_dim)(positions)
x = x + pos_embedding

attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
attn_out = layers.Dropout(0.1)(attn_out)
x = layers.LayerNormalization()(x + attn_out)

ff = layers.Dense(ff_dim, activation="relu")(x)
ff = layers.Dense(embed_dim)(ff)
ff = layers.Dropout(0.1)(ff)
x = layers.LayerNormalization()(x + ff)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

tiny_tf = Model(inputs, outputs)

tiny_tf.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(0.0005),
    metrics=["accuracy"]
)

tiny_tf.summary()

history = tiny_tf.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    verbose=1
)


Model: "functional_50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_23      │ (None, 60, 10)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 60, 16)    │        176 │ input_layer_23[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_19 (Add)        │ (None, 60, 16)    │          0 │ dense_42[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 16)    │      2,160 │ add_19[0][0],     │
│ (MultiHeadAttentio… │                   │            │ add_19[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_44          │ (None, 60, 16)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_20 (Add)        │ (None, 60, 16)    │          0 │ add_19[0][0],     │
│                     │                   │            │ dropout_44[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 16)    │         32 │ add_20[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 60, 32)    │        544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 60, 16)    │        528 │ dense_43[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_45          │ (None, 60, 16)    │          0 │ dense_44[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_21 (Add)        │ (None, 60, 16)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_45[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 16)    │         32 │ add_21[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_46          │ (None, 16)        │          0 │ global_average_p… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_45 (Dense)    │ (None, 1)         │         17 │ dropout_46[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,489 (13.63 KB)

 Trainable params: 3,489 (13.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 14s 68ms/step - accuracy: 0.2526 - loss: 0.8565 - val_accuracy: 0.6632 - val_loss: 0.6937
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6009 - loss: 0.7975 - val_accuracy: 0.7053 - val_loss: 0.5783
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6885 - loss: 0.6017 - val_accuracy: 0.7105 - val_loss: 0.5719
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6919 - loss: 0.5423 - val_accuracy: 0.7000 - val_loss: 0.6237
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6870 - loss: 0.5458 - val_accuracy: 0.7298 - val_loss: 0.6042
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6379 - loss: 0.6348 - val_accuracy: 0.6965 - val_loss: 0.6262
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6311 - loss: 0.7538 - val_accuracy: 0.7175 - val_loss: 0.5522
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7436 - loss: 0.6571 - val_accuracy: 0.7421 - val_los

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_prob = tiny_tf.predict(X_test)
threshold = 0.30
y_pred = (y_prob > threshold).astype(int)

print("Threshold:", threshold)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step
Threshold: 0.3
[[380 184]
 [  1   5]]
              precision    recall  f1-score   support

           0     0.9974    0.6738    0.8042       564
           1     0.0265    0.8333    0.0513         6

    accuracy                         0.6754       570
   macro avg     0.5119    0.7535    0.4278       570
weighted avg     0.9872    0.6754    0.7963       570



In [ ]:

# Build Functional Tiny Transformer with attention extraction


seq_len = X_train.shape[1]
feature_dim = X_train.shape[2]

embed_dim = 16
num_heads = 2
ff_dim = 32

inputs_f = layers.Input(shape=(seq_len, feature_dim))

# Dense projection
x_f = layers.Dense(embed_dim)(inputs_f)

# Positional embedding
positions_f = tf.range(start=0, limit=seq_len, delta=1)
pos_embedding_f = layers.Embedding(input_dim=seq_len, output_dim=embed_dim)(positions_f)
x_f = x_f + pos_embedding_f

# Multi-head attention WITH attention scores
attn_layer = tf.keras.layers.MultiHeadAttention(
    num_heads=num_heads,
    key_dim=embed_dim,
    dropout=0.1
)

attn_out_f, attn_scores_f = attn_layer(
    x_f, x_f, return_attention_scores=True
)

attn_out_f = layers.Dropout(0.1)(attn_out_f)
x_f = layers.LayerNormalization()(x_f + attn_out_f)

# Feedforward
ff_f = layers.Dense(ff_dim, activation="relu")(x_f)
ff_f = layers.Dense(embed_dim)(ff_f)
ff_f = layers.Dropout(0.1)(ff_f)
x_f = layers.LayerNormalization()(x_f + ff_f)

# Embedding vector
tf_embed = layers.GlobalAveragePooling1D()(x_f)
tf_embed_dropped = layers.Dropout(0.2)(tf_embed)

# Final probability output
tf_prob = layers.Dense(1, activation="sigmoid")(tf_embed_dropped)

# Functional transformer with 3 outputs
tiny_tf_func = Model(inputs_f, [tf_prob, tf_embed, attn_scores_f])

tiny_tf_func.summary()


Model: "functional_51"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26      │ (None, 60, 10)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_50 (Dense)    │ (None, 60, 16)    │        176 │ input_layer_26[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_26 (Add)        │ (None, 60, 16)    │          0 │ dense_50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ [(None, 60, 16),  │      2,160 │ add_26[0][0],     │
│ (MultiHeadAttentio… │ (None, 2, 60,     │            │ add_26[0][0]      │
│                     │ 60)]              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_51          │ (None, 60, 16)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_27 (Add)        │ (None, 60, 16)    │          0 │ add_26[0][0],     │
│                     │                   │            │ dropout_51[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 16)    │         32 │ add_27[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_51 (Dense)    │ (None, 60, 32)    │        544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_52 (Dense)    │ (None, 60, 16)    │        528 │ dense_51[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_52          │ (None, 60, 16)    │          0 │ dense_52[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_28 (Add)        │ (None, 60, 16)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_52[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 16)    │         32 │ add_28[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_53          │ (None, 16)        │          0 │ global_average_p… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_53 (Dense)    │ (None, 1)         │         17 │ dropout_53[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,489 (13.63 KB)

 Trainable params: 3,489 (13.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# Copy trained weights tiny_tf --> tiny_tf_func


for i in range(len(tiny_tf_func.layers)):
    try:
        tiny_tf_func.layers[i].set_weights(tiny_tf.layers[i].get_weights())
    except:
        pass  # Ignore mismatch layers such as attention score output


Extract probability, embedding, and attention scores

In [ ]:

# Extract probability, embedding, and attention scores

tf_prob, tf_embed, tf_attention = tiny_tf_func.predict(X_test)

tf_prob = tf_prob.flatten()
tf_flag = (tf_prob > 0.30).astype(int)

# Attention summary (single number per sample for JSON)
tf_attention_mean = tf_attention.mean(axis=(1,2,3))

print("tf_prob shape:", tf_prob.shape)
print("tf_embed shape:", tf_embed.shape)
print("tf_attention mean shape:", tf_attention_mean.shape)


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
tf_prob shape: (570,)
tf_embed shape: (570, 16)
tf_attention mean shape: (570,)


MERGING EXTRACTED OUPUT WITH DATASET WITH TIMESTAMP

In [ ]:
import pandas as pd
merged_clean_with_time = pd.read_csv("/content/drive/MyDrive/ds677/final_project/merged_clean_with_time.csv")


In [ ]:
seq_len = 60
timestamps = merged_clean_with_time["timestamp"].iloc[seq_len-1 : seq_len-1 + len(X_test)]
timestamps = timestamps.astype(str).tolist()

print(len(timestamps), len(X_test))


570 570


In [ ]:
records = []

for i in range(len(X_test)):
    rec = {
        "timestamp": timestamps[i],

        "lstm_prob": float(lstm_prob[i]),
        "lstm_flag": int(lstm_flag[i]),
        "lstm_hidden": lstm_hidden[i].tolist(),

        "tf_prob": float(tf_prob[i]),
        "tf_flag": int(tf_flag[i]),
        "tf_embed": tf_embed[i].tolist(),
        "tf_attention_mean": float(tf_attention_mean[i])
    }

    records.append(rec)

print("Total records:", len(records))


Total records: 570


CREATING JSON FILES FOR LLM TRAINING

In [ ]:
project_knowledge = {
    "project_name": "Hybrid LSTM + Transformer Chiller Anomaly System",
    "sequence_length": seq_len,
    "num_features": X_train.shape[2],
    "num_records": len(records),
    "records": records
}

import json
with open("project_knowledge.json", "w") as f:
    json.dump(project_knowledge, f, indent=2)

print("Created: project_knowledge.json")


Created: project_knowledge.json


In [ ]:
import json

BASE = "/content/drive/MyDrive/ds677/final_project/"

with open(BASE + "anomaly_timeline.json") as f:
    anomaly_timeline = json.load(f)

with open(BASE + "feature_drift_patterns.json") as f:
    feature_drift = json.load(f)

with open(BASE + "sensor_statistics.json") as f:
    sensor_stats = json.load(f)

with open(BASE + "engineering_formulas.json") as f:
    eng_formulas = json.load(f)

domain_knowledge = {
    "anomaly_timeline": anomaly_timeline,
    "feature_drift_patterns": feature_drift,
    "sensor_statistics": sensor_stats,
    "engineering_formulas": eng_formulas
}

print("domain_knowledge loaded.")


domain_knowledge loaded.


In [ ]:
with open("project_knowledge.json") as f:
    dl_outputs = json.load(f)


In [ ]:
master_knowledge = {
    "project_name": "Chiller AI Reasoning System",

    "domain_knowledge": domain_knowledge,

    "deep_learning_outputs": dl_outputs,

    "notes": "Merged domain physics + anomaly logs + LSTM/Transformer latent states."
}

with open("master_knowledge.json", "w") as f:
    json.dump(master_knowledge, f, indent=2)

print("Saved: master_knowledge.json")


Saved: master_knowledge.json


In [ ]:
with open(BASE + "master_knowledge.json", "w") as f:
    json.dump(master_knowledge, f, indent=2)

print("master_knowledge.json saved to Drive!")


master_knowledge.json saved to Drive!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import json

BASE = "/content/drive/MyDrive/ds677/final_project/"

with open(BASE + "master_knowledge.json") as f:
    master_knowledge = json.load(f)

print("Master file loaded. Ready for LLM.")


Mounted at /content/drive
Master file loaded. Ready for LLM.


In [ ]:
def compress_knowledge(KNOWLEDGE):

    domain = KNOWLEDGE["domain_knowledge"]
    dl = KNOWLEDGE["deep_learning_outputs"]

    compressed = {
        "project_name": KNOWLEDGE["project_name"],

        "anomaly_summary": {
            "total_anomalies": domain["anomaly_timeline"].get("total_anomalies"),
            "example_timestamps": domain["anomaly_timeline"].get("anomaly_timestamps", [])[:5]
        },

        "top_drift_features": sorted(
            domain["feature_drift_patterns"],
            key=domain["feature_drift_patterns"].get,
            reverse=True
        )[:5],

        "sensor_statistics": domain["sensor_statistics"],
        "engineering_formulas": domain["engineering_formulas"],

        "dl_summary": {
            "num_records": dl.get("num_records"),
            "example_record": dl["records"][0] if dl.get("records") else None
        }
    }

    return compressed


In [ ]:
COMPRESSED = compress_knowledge(master_knowledge)
print("Compression complete.")


Compression complete.


LLM IMPLEMENT

In [ ]:
import requests
import json

GROQ_API_KEY = ""

def build_prompt(question):
    comp = json.dumps(COMPRESSED, indent=2)
    return f"""
You are an expert chiller anomaly engineer.
Use ONLY this compressed knowledge while answering.

COMPRESSED KNOWLEDGE:
{comp}

QUESTION:
{question}
"""

def get_llm_response(question):
    payload = {
        "model": "llama-3.1-8b-instant",
        "messages": [
            {"role": "user", "content": build_prompt(question)}
        ]
    }

    headers = {"Authorization": f"Bearer {GROQ_API_KEY}"}
    r = requests.post("https://api.groq.com/openai/v1/chat/completions",
                      json=payload, headers=headers)

    out = r.json()
    return out["choices"][0]["message"]["content"]


In [ ]:
print(get_llm_response(build_prompt("how much temperature to maintain for optimum efficiency during day time")))


Based on the compressed knowledge provided, we can analyze the sensor statistics for optimal temperature settings during day time.

From the sensor statistics, the "Chiller 6 Supply Temperature" has a mean value of 60.89662451580723 and a standard deviation of 38.051841876887096. During anomalies, this temperature drops to a mean value of 34.05150403239879.

For optimal efficiency, I recommend maintaining a supply temperature between 55°F and 65°F (13°C and 18°C) during day time. This temperature range is suitable for most chillers and can help to achieve efficient operation while minimizing energy consumption.

However, please note that the exact optimal temperature setting may depend on various factors such as the specific chiller model, ambient temperature, and load requirements. It is essential to consult the chiller's manufacturer manual and perform on-site testing to determine the optimal temperature setting for your specific system.

References:
- Cooling_Load_formula
- Temperat

In [ ]:
question = "When did anomalies occur in this chiller?"
final_prompt = build_prompt(question)
print(get_llm_response(final_prompt))


Groq API Error: {'error': {'message': 'Request too large for model `llama-3.1-8b-instant` in organization `org_01kbxz8fqkfr8rrj0jbwsw2283` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 27605, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
None


In [ ]:
print(get_llm_response("what is chiller"))


A chiller is a type of heat transfer device that is used to cool a liquid, typically water or a water-glycol mixture, by transferring heat from the liquid to a refrigerant, which is then dissipated to the surrounding environment.

Chillers are commonly used in various applications, including:

1. **Air conditioning**: Chillers are used to cool air in buildings, industrial processes, and data centers.
2. **Industrial processes**: Chillers are used in various industrial processes, such as textile production, food processing, and chemical manufacturing.
3. **Data centers**: Chillers are used to cool the servers and other equipment in data centers.
4. **Medical facilities**: Chillers are used in hospitals and medical facilities to cool equipment and maintain a consistent temperature.
5. **Food and beverage**: Chillers are used in the food and beverage industry to cool beverages, such as beer and soft drinks, and to maintain a consistent temperature in food processing areas.

Chillers work 